<a href="https://colab.research.google.com/github/victrixgyasi/AVeriTeC_shared_task/blob/main/BART_BASE_finetuned_FOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install accelerate nltk numpy tqdm torch torchvision transformers peft subprocess32==3.5.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_c

In [ ]:
import numpy as np
import json
import nltk
import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from accelerate import Accelerator
from transformers import BartForConditionalGeneration
import nltk
nltk.download('punkt')
nltk.download('wordnet')
# Mount on GDrive
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Mounted at /content/drive


## **Finetuning on claims and predicates only**

In [ ]:
def open_data_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        #print(data)
    return data

def get_data(file_name):
    examples = open_data_file(file_name)
    claim_predicates = []

    for example in examples:
        claim = example["claim"]
        predicates = [q for q in example["predicates"]]

        claim_dict_builder = {}
        claim_dict_builder["claim"] = claim
        claim_dict_builder["predicates"] = predicates

        claim_predicates.append(claim_dict_builder)
    return claim_predicates

In [ ]:
model_load_path = '/content/drive/My Drive/Colab Notebooks/Fact-checking explanations/AVeriTeC/data/results/trained_model_100_FOL'
model = BartForConditionalGeneration.from_pretrained(model_load_path)
tokenizer = BartTokenizer.from_pretrained(model_load_path)

In [ ]:
def get_claims(file_name):
    examples = open_data_file(file_name)
    claims = []
    for example in examples:
        claim = example["claim"]
        questions = [] # empty
        claim_dict_builder = {}
        claim_dict_builder["claim"] = claim
        claim_dict_builder["questions"] = questions
        claims.append(claim_dict_builder)
    return claims

def write_to_json_file(output_path:str, data, append = True):
    """
    Write data to a JSON file. If the file exists, it will be overwritten.

    Parameters:
    filename (str): The name of the file where the JSON data will be written.
    data (dict): The data to write in JSON format.

    """
    # Open the file in append mode ('a') if append is True, otherwise in write mode ('w')
    mode = 'a' if append else 'w'
    with open(output_path, mode) as file:
        # Write the data to the file in JSON format
        json.dump(data, file, indent=4)

def get_json_results(input_file_name: str, output_path:str, append = True):
    claims = get_claims(input_file_name)
    for example in claims:
        example["predicates"] = []  # Initialize the list if not already present
        for i in range(5):
            claim = example["claim"]

            # Tokenize the input text
            input_encodings = tokenizer(
                claim, max_length=512, truncation=True, padding="max_length", return_tensors="pt"
            )

            input_ids = input_encodings.input_ids
            attention_mask = input_encodings.attention_mask

            # Generate predictions with sampling
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=512,
                num_return_sequences=1,
                do_sample=True,  # Enable sampling
                top_k=50,  # Use top-k sampling
                top_p=0.95,  # Use top-p (nucleus) sampling
                temperature=1.5,  # Adjust temperature for more diversity
                num_beams=5,  # Use beam search
                early_stopping=True
            )

            # Append the decoded generate predicate to the list
            example["predicates"].append(tokenizer.decode(outputs[0], skip_special_tokens=True))

        # Print the example to verify the generated questions
        print(example)
    write_to_json_file(output_path, claims, append = True)


input_file_name = '/content/drive/MyDrive/Colab Notebooks/Fact-checking explanations/Refined Code/dev.json'
output_path = '/content/drive/MyDrive/Colab Notebooks/Fact-checking explanations/Refined Code/10q, misinfo included/FOL_predicates.json'
get_json_results(input_file_name, output_path, append = True)
print("Completed.")

{'claim': 'In a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'questions': [], 'predicates': ['DeclinedToAppell(Sean Connery, apple commercial) - Verify that in a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'Reflected(Sean Connery, apple commercial) - Verify that in a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'Resigned(Sean Connery, apple commercial) - Verify that in a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'Resigned(Sean Connery, apple commercial) - Verify that in a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'Refested(Sean Connery, apple commercial) - Verify that in a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.']}
{'claim': 'Trump Administration claimed songwriter Billie Eilish Is Destroying Our Country In Leaked Documents', 'questions': [], 'predicates': ['Claimed(Trump Adm

## **Finetuning on predicates and questions only**

In [ ]:
model_load_path = '/content/drive/My Drive/Colab Notebooks/Fact-checking explanations/AVeriTeC/data/results/trained_model_151_FOL_questions_v2'
model = BartForConditionalGeneration.from_pretrained(model_load_path)
tokenizer = BartTokenizer.from_pretrained(model_load_path)

In [ ]:
# So we can generate questions from claims and predicates
def get_claims_predicates(file_name):
    examples = open_data_file(file_name)
    claims = []
    for example in examples:
        claim = example["claim"]
        predicates = example["predicates"]
        questions = [] # empty
        claim_dict_builder = {}
        claim_dict_builder["claim"] = claim
        claim_dict_builder["predicates"] = predicates
        claim_dict_builder["questions"] = questions
        claims.append(claim_dict_builder)
    return claims

def write_to_json_file(output_path:str, data, append = True):
    """
    Write data to a JSON file. If the file exists, it will be overwritten.

    Parameters:
    filename (str): The name of the file where the JSON data will be written.
    data (dict): The data to write in JSON format.

    """
    # Open the file in append mode ('a') if append is True, otherwise in write mode ('w')
    mode = 'a' if append else 'w'
    with open(output_path, mode) as file:
        # Write the data to the file in JSON format
        json.dump(data, file, indent=4)

def get_json_results(input_file_name: str, output_path:str, append = True):
    claims = get_claims_predicates(input_file_name)
    for example in claims:
        example["questions"] = []  # Initialize the list if not already present
        for i in range(10):
            claim = example["claim"]
            predicates = example["predicates"]

            # Tokenize the input text
            input_encodings = tokenizer(
                predicates, max_length=512, truncation=True, padding="max_length", return_tensors="pt"
            ) # get questions based on predicates

            input_ids = input_encodings.input_ids
            attention_mask = input_encodings.attention_mask

            # Generate predictions with sampling
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=512,
                num_return_sequences=1,
                do_sample=True,  # Enable sampling
                top_k=50,  # Use top-k sampling
                top_p=0.95,  # Use top-p (nucleus) sampling
                temperature=1.5,  # Adjust temperature for more diversity
                num_beams=5,  # Use beam search
                early_stopping=True
            )

            # Decode the output
            question_dict = {"question": tokenizer.decode(outputs[0], skip_special_tokens=True)}

            # Append the generated question to the list
            example["questions"].append(question_dict)

        # Print the example to verify the generated questions
        print(example)
    write_to_json_file(output_path, claims, append = True)




input_file_name = '/content/drive/MyDrive/Colab Notebooks/Fact-checking explanations/Refined Code/10q, misinfo included/FOL_predicates.json'
output_path = '/content/drive/MyDrive/Colab Notebooks/Fact-checking explanations/Refined Code/10q, misinfo included/FOL_question.json'
get_json_results(input_file_name, output_path, append = True)
print("Completed.")

{'claim': 'In a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'predicates': ['DeclinedToAppell(Sean Connery, apple commercial) - Verify that in a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'Reflected(Sean Connery, apple commercial) - Verify that in a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'Resigned(Sean Connery, apple commercial) - Verify that in a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'Resigned(Sean Connery, apple commercial) - Verify that in a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'Refested(Sean Connery, apple commercial) - Verify that in a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.'], 'questions': [{'question': 'Did Sean Connery refuse to appear in an apple commercial?'}, {'question': 'Did Sean Connery refuse to appear in an apple commercial?'}, {'question': 'Di

In [ ]:
!pip install leven

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 6.0 MB/s eta 0:00:00
  Created wheel for leven: filename=leven-1.0.4-cp310-cp310-linux_x86_64.whl size=64148 sha256=82b0f78b9bc53459ef7db13310177e8314d4aed203d6faa377d800944156bda0
  Stored in directory: /root/.cache/pip/wheels/a8/c7/c4/99e93cc37bb627f3d4d153d1a62feb3491c2e832a33e3ea2dc
Successfully built leven


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/AVeriTeC')
!python 'drive/My Drive/Colab Notebooks/Fact-checking explanations/AVeriTeC/FOL_eval.py' --predictions '/content/drive/My Drive/Colab Notebooks/Fact-checking explanations/AVeriTeC/data/results/BART_BASE_result_151_FOL_questions.json' --references '/content/drive/My Drive/Colab Notebooks/Fact-checking explanations/AVeriTeC/data/dev.json'

AVeriTeC evaluation:
Question-only score (HU-meteor):        0.35313239202414265


In [ ]:
# BART-BASE finetuned with training data claims and questions: HU-Meteor0.37336755684564005